In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

# Get Constituents from 2010


In [75]:
constituents = pd.read_csv("../../data/SP500_constituents.csv")
constituents = constituents[["PERMNO", "from", "thru"]]
constituents["from"] = pd.to_datetime(constituents["from"])
constituents["thru"] = pd.to_datetime(constituents["thru"])

constituents = constituents[constituents["thru"] >= "2010-01-01"]

constituents.info()
constituents.to_csv("sp500_constituents_2010.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 811 entries, 1 to 1321
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   PERMNO  811 non-null    int64         
 1   from    811 non-null    datetime64[ns]
 2   thru    811 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 25.3 KB


In [76]:
constituents["PERMNO"].to_csv(
    "sp500_permnos_2010.csv",
    index=False,
)

# Process Price Data


## Load Price Data & Basic Transformations


In [3]:
prices = pd.read_csv("../../data/prices_data.csv", parse_dates=["YYYYMMDD"])
prices: pd.DataFrame

prices = prices.drop(
    [
        "SecInfoStartDt",
        "SecInfoEndDt",
        "SecurityBegDt",
        "SecurityEndDt",
        "ICBIndustry",
        "SecurityNm",
        "DlyPrevCap",
        "DlyPrevCapFlg",
        "DlyPrevPrc",
        "DlyPrevPrcFlg",
        "DlyPrevDt",
    ],
    axis=1,
)

In [4]:
# renaming columns to sensible values
prices.rename(
    columns={
        "YYYYMMDD": "date",
        "PERMNO": "permno",
        "PERMCO": "permco",
        "Ticker": "ticker",
    },
    inplace=True,
)

In [5]:
prices.isna().sum()

permno             0
ticker          1418
permco             0
SICCD              0
NAICS              0
date               0
DlyCalDt           0
DlyPrc          1173
DlyPrcFlg          0
DlyCap          1473
DlyCapFlg          0
DlyRetx         1443
DlyVol          1474
DlyClose       14731
DlyLow         14731
DlyHigh        14731
DlyBid         57383
DlyAsk         57383
DlyOpen        19808
DlyNumTrd    2970055
DlyPrcVol       1475
ShrOut           300
dtype: int64

In [6]:
# there are some duplicate dates for the same permno, here I remove them from the df
prices = prices.drop_duplicates(subset=["permno", "date"])

# checking that there are no duplicates left
for permno in prices["permno"].unique():
    dates = prices[prices["permno"] == permno]["date"]

    if dates.shape[0] != dates.nunique():
        print(permno)
        print(dates)
        # prices[prices["permno"] == permno].to_excel('debug.xlsx')
        print(dates[dates.duplicated()])
        break

In [7]:
# calculating returns
prices.sort_values(["permno", "date"], inplace=True)
prices["return"] = prices["DlyPrc"].pct_change()
prices["vol_36mo"] = prices["return"].rolling(window=252 * 3).std()
prices["vol_252d"] = prices["return"].rolling(window=252).std()
prices["vol_126d"] = prices["return"].rolling(window=126).std()
prices["vol_21d"] = prices["return"].rolling(window=21).std()

# to make sure that the calculation does count different tickers (when the dataframe changes from one ticker to another)
mask = prices.permno != prices.permno.shift(1)
prices.loc[mask, "return"] = np.nan

C:\Users\andre\AppData\Local\Temp\ipykernel_26528\82406667.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  prices["return"] = prices["DlyPrc"].pct_change()


In [8]:
prices

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,DlyAsk,DlyOpen,DlyNumTrd,DlyPrcVol,ShrOut,return,vol_36mo,vol_252d,vol_126d,vol_21d
0,10078,SUNW,8021,3570,0,2000-01-03,2000-01-03,76.500,TR,1.194246e+08,...,76.5000,79.2500,26427.0,1.168195e+09,1561106.0,NaN,NaN,NaN,NaN,NaN
1,10078,SUNW,8021,3570,0,2000-01-04,2000-01-04,71.750,TR,1.120094e+08,...,71.7500,73.4375,32108.0,1.513832e+09,1561106.0,-0.062092,NaN,NaN,NaN,NaN
2,10078,SUNW,8021,3570,0,2000-01-05,2000-01-05,71.875,TR,1.122045e+08,...,71.9375,70.1250,49741.0,2.161515e+09,1561106.0,0.001742,NaN,NaN,NaN,NaN
3,10078,SUNW,8021,3570,0,2000-01-06,2000-01-06,68.000,TR,1.061552e+08,...,68.0000,70.4375,30663.0,1.546456e+09,1561106.0,-0.053913,NaN,NaN,NaN,NaN
4,10078,SUNW,8021,3570,0,2000-01-07,2000-01-07,71.875,TR,1.122045e+08,...,71.8750,67.1875,34162.0,1.469862e+09,1561106.0,0.056985,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983884,93436,TSLA,53453,9999,336110,2023-12-22,2023-12-22,252.540,TR,8.028047e+08,...,252.5400,256.7600,972876.0,2.352373e+10,3178921.0,-0.007701,0.044308,0.035495,0.030185,0.020859
3983885,93436,TSLA,53453,9999,336110,2023-12-26,2023-12-26,256.610,TR,8.157429e+08,...,256.7100,254.4900,954492.0,2.224827e+10,3178921.0,0.016116,0.044308,0.035025,0.030031,0.021031
3983886,93436,TSLA,53453,9999,336110,2023-12-27,2023-12-27,261.440,TR,8.310971e+08,...,261.5600,258.3500,1046971.0,2.777820e+10,3178921.0,0.018822,0.044313,0.035013,0.030002,0.021262
3983887,93436,TSLA,53453,9999,336110,2023-12-28,2023-12-28,253.180,TR,8.048392e+08,...,253.1900,263.6600,1166971.0,2.867281e+10,3178921.0,-0.031594,0.044319,0.034286,0.030136,0.020625


In [9]:
# normalize returns dividing by volatility
VOLATILITY_WINDOW = 252
prices["vol_adjusted_return"] = prices["return"] / prices[f"vol_{VOLATILITY_WINDOW}d"]

In [10]:
prices.tail()

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,DlyOpen,DlyNumTrd,DlyPrcVol,ShrOut,return,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return
3983884,93436,TSLA,53453,9999,336110,2023-12-22,2023-12-22,252.54,TR,8.028047e+08,...,256.76,972876.0,2.352373e+10,3178921.0,-0.007701,0.044308,0.035495,0.030185,0.020859,-0.216973
3983885,93436,TSLA,53453,9999,336110,2023-12-26,2023-12-26,256.61,TR,8.157429e+08,...,254.49,954492.0,2.224827e+10,3178921.0,0.016116,0.044308,0.035025,0.030031,0.021031,0.460136
3983886,93436,TSLA,53453,9999,336110,2023-12-27,2023-12-27,261.44,TR,8.310971e+08,...,258.35,1046971.0,2.777820e+10,3178921.0,0.018822,0.044313,0.035013,0.030002,0.021262,0.537581
3983887,93436,TSLA,53453,9999,336110,2023-12-28,2023-12-28,253.18,TR,8.048392e+08,...,263.66,1166971.0,2.867281e+10,3178921.0,-0.031594,0.044319,0.034286,0.030136,0.020625,-0.921481
3983888,93436,TSLA,53453,9999,336110,2023-12-29,2023-12-29,248.48,TR,7.898983e+08,...,255.10,1062240.0,2.492781e+10,3178921.0,-0.018564,0.044324,0.034266,0.030147,0.020932,-0.541761


## Calculating Industry Specific Returns


In [11]:
prices.head()

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,DlyOpen,DlyNumTrd,DlyPrcVol,ShrOut,return,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return
0,10078,SUNW,8021,3570,0,2000-01-03,2000-01-03,76.500,TR,1.194246e+08,...,79.2500,26427.0,1.168195e+09,1561106.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10078,SUNW,8021,3570,0,2000-01-04,2000-01-04,71.750,TR,1.120094e+08,...,73.4375,32108.0,1.513832e+09,1561106.0,-0.062092,NaN,NaN,NaN,NaN,NaN
2,10078,SUNW,8021,3570,0,2000-01-05,2000-01-05,71.875,TR,1.122045e+08,...,70.1250,49741.0,2.161515e+09,1561106.0,0.001742,NaN,NaN,NaN,NaN,NaN
3,10078,SUNW,8021,3570,0,2000-01-06,2000-01-06,68.000,TR,1.061552e+08,...,70.4375,30663.0,1.546456e+09,1561106.0,-0.053913,NaN,NaN,NaN,NaN,NaN
4,10078,SUNW,8021,3570,0,2000-01-07,2000-01-07,71.875,TR,1.122045e+08,...,67.1875,34162.0,1.469862e+09,1561106.0,0.056985,NaN,NaN,NaN,NaN,NaN


In [12]:
# define the part of the code to use - in this case 2 digits
DIGITS = 2

prices["NAICS"] = prices["NAICS"].replace(0, np.nan)  # replace 0s with NaNs
prices["naics_processed"] = prices["NAICS"]

mask = prices["naics_processed"].notna()
prices.loc[mask, "naics_processed"] = prices.loc[mask, "naics_processed"].apply(
    lambda x: float(str(x)[:DIGITS])
)
prices["naics_processed"].unique()

array([nan, 33., 51., 54., 22., 52., 42., 31., 32., 56., 44., 72., 62.,
       45., 55., 48., 21., 53., 81., 11., 23., 71., 61., 49., 92.])

In [13]:
# calculate industry returns

USE_VOL_ADJUSTED_RETURNS = True
return_col = (
    "vol_adjusted_return" if USE_VOL_ADJUSTED_RETURNS else "return"
)  # choose whether to use vol adjusted returns or not

# create df with dates as index and industry codes as columns
industry_returns_df = pd.DataFrame(
    columns=prices["naics_processed"].unique(), index=prices.date.unique()
)

# do not calculate mean when the code is NaN
# industry_returns_df.drop(np.nan, axis=1, inplace=True)

# for each industry code calculate the average industry return at each date
for industry_code in industry_returns_df.columns:
    if industry_code == np.nan:
        continue
    # print(f'calculating for industry code {industry_code}')

    # filter companies in the industry
    companies = prices[prices["naics_processed"] == industry_code]

    # groupby the date and calculate the weighted average return (market-cap weighted)
    industry_return = companies.groupby("date").apply(
        lambda x: np.average(x[return_col], weights=x["DlyCap"])
    )
    industry_returns_df.loc[industry_return.index, industry_code] = industry_return

industry_returns_df

C:\Users\andre\AppData\Local\Temp\ipykernel_26528\1793762166.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  industry_return = companies.groupby("date").apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26528\1793762166.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  industry_return = companies.groupby("date").apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26528\1793762166.py:26: DeprecationW

,NaN,33.0,51.0,54.0,22.0,52.0,42.0,31.0,32.0,56.0,...,48.0,21.0,53.0,81.0,11.0,23.0,71.0,61.0,49.0,92.0
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,NaN,0.002741,0.091709,0.360755,0.282013,0.195201,0.274217,-0.199422,0.451904,0.049808,...,0.292258,0.258227,0.07307,NaN,0.224427,0.04018,-0.197358,0.660273,0.324253,0.598981
2023-12-26,NaN,0.312875,0.047846,0.079546,0.452064,0.260824,0.259827,0.381924,0.179273,0.215332,...,0.256515,0.528816,0.489693,NaN,0.709837,0.30445,0.274026,0.299218,0.383878,0.154115
2023-12-27,NaN,0.104966,0.003933,-0.271957,-0.082728,0.17421,0.025324,0.175276,0.263092,0.064598,...,0.059942,-0.174847,0.22308,NaN,-0.02601,0.34479,-0.212496,0.100052,-0.206631,-0.045553
2023-12-28,NaN,-0.000643,0.157919,-0.034475,0.42022,0.229039,-0.192061,0.232939,-0.113856,0.343173,...,0.013811,-0.636582,0.247193,NaN,0.143162,-0.051601,0.562177,-0.297958,0.34585,0.068404


In [14]:
# see how many NaN values are there in the industry returns
industry_returns_df.T.isna().sum()[industry_returns_df.T.isna().sum() > 4]

2000-01-03    25
2000-01-04    25
2000-01-05    25
2000-01-06    25
2000-01-07    25
              ..
2020-11-06     5
2020-11-17     5
2020-12-10     5
2020-12-15     5
2021-06-03     5
Length: 4090, dtype: int64

In [15]:
# calculate industry-relative returns

# map each datapoint to its industry return
for industry_code in industry_returns_df.columns.drop(np.nan):

    # create mask for the companies belonging to the industry
    mask = prices["naics_processed"] == industry_code

    # map the industry return to the companies at each date
    prices.loc[mask, "industry_return"] = prices.loc[mask, "date"].map(
        industry_returns_df[industry_code]
    )

    # DEBUG code: the industry return for a specific date should be the same for all companies belonging to the same industry
    # print(prices.loc[mask, ['date', 'naics_processed', 'ticker', 'industry_return']].sort_values('date'))

prices["industry_relative_return"] = (
    prices[return_col] - prices["industry_return"]
).astype(
    "float64"
)  # from above, use vol adjusted if USE_VOL_ADJUSTED_RETURNS is True

prices

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,ShrOut,return,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return,naics_processed,industry_return,industry_relative_return
0,10078,SUNW,8021,3570,NaN,2000-01-03,2000-01-03,76.500,TR,1.194246e+08,...,1561106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10078,SUNW,8021,3570,NaN,2000-01-04,2000-01-04,71.750,TR,1.120094e+08,...,1561106.0,-0.062092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10078,SUNW,8021,3570,NaN,2000-01-05,2000-01-05,71.875,TR,1.122045e+08,...,1561106.0,0.001742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10078,SUNW,8021,3570,NaN,2000-01-06,2000-01-06,68.000,TR,1.061552e+08,...,1561106.0,-0.053913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10078,SUNW,8021,3570,NaN,2000-01-07,2000-01-07,71.875,TR,1.122045e+08,...,1561106.0,0.056985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983884,93436,TSLA,53453,9999,336110.0,2023-12-22,2023-12-22,252.540,TR,8.028047e+08,...,3178921.0,-0.007701,0.044308,0.035495,0.030185,0.020859,-0.216973,33.0,0.002741,-0.219714
3983885,93436,TSLA,53453,9999,336110.0,2023-12-26,2023-12-26,256.610,TR,8.157429e+08,...,3178921.0,0.016116,0.044308,0.035025,0.030031,0.021031,0.460136,33.0,0.312875,0.147261
3983886,93436,TSLA,53453,9999,336110.0,2023-12-27,2023-12-27,261.440,TR,8.310971e+08,...,3178921.0,0.018822,0.044313,0.035013,0.030002,0.021262,0.537581,33.0,0.104966,0.432614
3983887,93436,TSLA,53453,9999,336110.0,2023-12-28,2023-12-28,253.180,TR,8.048392e+08,...,3178921.0,-0.031594,0.044319,0.034286,0.030136,0.020625,-0.921481,33.0,-0.000643,-0.920838


In [16]:
# see correlation between columns
prices.dropna(subset=["industry_relative_return"]).drop(
    [
        "permno",
        "permco",
        "SICCD",
        "NAICS",
        "DlyCap",
        "ShrOut",
        "naics_processed",
        "ticker",
    ],
    axis=1,
).corr(numeric_only=True)

,DlyPrc,DlyRetx,DlyVol,DlyClose,DlyLow,DlyHigh,DlyBid,DlyAsk,DlyOpen,DlyNumTrd,DlyPrcVol,return,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return,industry_relative_return
DlyPrc,1.000000,0.000128,-0.011485,1.000000,0.999973,0.999972,0.999998,0.999976,0.999948,0.113792,0.027537,-0.000006,-0.001880,-0.001248,-0.001035,-0.000697,0.000918,0.001114
DlyRetx,0.000128,1.000000,0.008956,0.000140,0.000011,0.000008,0.000103,0.000106,-0.000113,-0.000533,0.009822,0.270645,0.000767,0.000494,0.001202,0.001702,0.850092,0.583418
DlyVol,-0.011485,0.008956,1.000000,-0.011521,-0.011521,-0.011515,-0.011475,-0.011476,-0.011517,0.757045,0.356478,0.003043,-0.005902,-0.001048,0.000354,0.002900,-0.005156,-0.000917
DlyClose,1.000000,0.000140,-0.011521,1.000000,0.999973,0.999972,0.999998,0.999976,0.999948,0.113456,0.027509,-0.000004,-0.001941,-0.001309,-0.001133,-0.000808,0.000921,0.001120
DlyLow,0.999973,0.000011,-0.011521,0.999973,1.000000,0.999958,0.999974,0.999950,0.999971,0.111984,0.027456,-0.000033,-0.001941,-0.001310,-0.001135,-0.000810,0.000723,0.000998
DlyHigh,0.999972,0.000008,-0.011515,0.999972,0.999958,1.000000,0.999971,0.999951,0.999975,0.115044,0.027571,-0.000033,-0.001941,-0.001308,-0.001132,-0.000806,0.000707,0.001005
DlyBid,0.999998,0.000103,-0.011475,0.999998,0.999974,0.999971,1.000000,0.999976,0.999948,0.113821,0.027546,-0.000011,-0.001883,-0.001251,-0.001040,-0.000696,0.000863,0.001042
DlyAsk,0.999976,0.000106,-0.011476,0.999976,0.999950,0.999951,0.999976,1.000000,0.999927,0.113753,0.027545,-0.000011,-0.001880,-0.001249,-0.001038,-0.000696,0.000863,0.001044
DlyOpen,0.999948,-0.000113,-0.011517,0.999948,0.999971,0.999975,0.999948,0.999927,1.000000,0.113678,0.027515,-0.000060,-0.001941,-0.001309,-0.001133,-0.000808,0.000525,0.000879
DlyNumTrd,0.113792,-0.000533,0.757045,0.113456,0.111984,0.115044,0.113821,0.113753,0.113678,1.000000,0.863182,-0.000695,0.019050,0.017073,0.014374,0.016186,-0.012347,-0.000914


# Fama-French Residual Returns


In [17]:
ff_data = pd.read_csv(
    "../../data/fama_french_5_factors.csv", parse_dates=["date"], date_format="%Y%m%d"
)

ff_data = ff_data.set_index("date").reindex(prices.date.unique())
ff_data

,Mkt-RF,SMB,HML,RMW,CMA,RF
date,,,,,,
2000-01-03,-0.71,-0.06,-1.41,-1.50,-0.64,0.021
2000-01-04,-4.06,0.33,2.06,0.47,1.45,0.021
2000-01-05,-0.09,0.33,0.16,0.41,1.11,0.021
2000-01-06,-0.73,-0.04,1.26,0.65,1.21,0.021
2000-01-07,3.21,-0.94,-1.42,-0.88,-0.96,0.021
...,...,...,...,...,...,...
2023-12-22,0.21,0.61,0.09,-0.64,0.19,0.021
2023-12-26,0.48,0.81,0.46,-0.34,-0.15,0.021
2023-12-27,0.16,0.16,0.12,-0.31,-0.14,0.021


In [18]:
REGRESSION_WINDOW = 1 * 252  # in the paper 36months
MIN_OBS = 126


prices_filtered = prices[["permno", "date", "ticker", "return", "vol_36mo"]]


# prices_filtered.loc[:, "return"] *= 100  # convert to percentage


def perform_rolling_ols(permno):

    stock_data = prices_filtered[prices_filtered["permno"] == permno]

    if stock_data.shape[0] < REGRESSION_WINDOW:
        print("returning all NaNs")
        return None

    # print(stock_data)

    ff_factors = ff_data.reindex(stock_data["date"])

    data = stock_data.merge(ff_factors, left_on="date", right_index=True)

    endog = data["return"] - data["RF"]

    exog = sm.add_constant(data[["Mkt-RF", "SMB", "HML", "RMW", "CMA"]])

    rolling_ols = RollingOLS(
        endog, exog, window=REGRESSION_WINDOW, min_nobs=MIN_OBS, expanding=True
    )

    rres = rolling_ols.fit(params_only=True)

    # calculate pred values by multiplying the exog matrix with the params matrix

    # the correct predictions (the ones calculated using the factors and params at the same date) lie on the diagonal of the matrix

    # to get e we do y - the diagonal of the pred matrix

    pred: np.ndarray = (exog @ rres.params.T).to_numpy()

    e = endog - pred.diagonal()

    scaled_e = e / data["vol_36mo"]  # scale by volatility

    scaled_e.index = data["date"]

    return scaled_e

    # scale by volatility


prices["str_paper_residuals"] = np.nan


for permno in prices["permno"].unique():
    print(f"Regressing Permno {permno}")

    # print(prices.loc[mask, "str_paper_residuals"])

    scaled_e = perform_rolling_ols(permno)
    if scaled_e is None:
        continue

    # print("scaled_e")

    # print(scaled_e)

    mask = (prices["permno"] == permno) & (prices["date"].isin(scaled_e.index))

    # print(prices[mask].shape, scaled_e.shape)

    masked = prices[mask]

    # print(masked["date"].shape, masked["date"].nunique())

    prices.loc[mask, "str_paper_residuals"] = scaled_e

    # find out how to merge correctly in the main dataframe efficiently

    # prices.loc[mask, "str_paper_residuals"] = scaled_e

Regressing Permno 10078
Regressing Permno 10104
Regressing Permno 10107
Regressing Permno 10137
Regressing Permno 10138
Regressing Permno 10145
Regressing Permno 10147
Regressing Permno 10225
Regressing Permno 10299
Regressing Permno 10324
Regressing Permno 10516
Regressing Permno 10696
Regressing Permno 10909
Regressing Permno 11081
Regressing Permno 11308
Regressing Permno 11403
Regressing Permno 11404
Regressing Permno 11533
Regressing Permno 11552
Regressing Permno 11600
Regressing Permno 11618
Regressing Permno 11674
Regressing Permno 11703
Regressing Permno 11754
Regressing Permno 11762
Regressing Permno 11786
Regressing Permno 11850
Regressing Permno 11891
Regressing Permno 11896
Regressing Permno 11955
Regressing Permno 11976
Regressing Permno 12052
Regressing Permno 12060
Regressing Permno 12062
Regressing Permno 12067
Regressing Permno 12073
Regressing Permno 12076
Regressing Permno 12084
Regressing Permno 12140
Regressing Permno 12308
Regressing Permno 12345
Regressing Permn

KeyboardInterrupt: 

In [ ]:
prices[prices["a"] == 12346]

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,DlyNumTrd,DlyPrcVol,ShrOut,return,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return,str_paper_residuals
203169,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-14,2010-10-14,27.07,TR,5868018.04,...,NaN,20085940.0,216772.0,NaN,0.032073,0.047258,0.064950,0.156372,NaN,NaN
203170,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-15,2010-10-15,26.66,TR,5779141.52,...,NaN,15884028.0,216772.0,-0.015146,0.032070,0.047264,0.064926,0.156192,-0.320451,NaN
203171,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-18,2010-10-18,26.75,TR,5798651.00,...,NaN,11890375.0,216772.0,0.003376,0.032070,0.047258,0.064930,0.156359,0.071435,NaN
203172,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-19,2010-10-19,26.46,TR,5735787.12,...,NaN,22451310.0,216772.0,-0.010841,0.032059,0.047234,0.064908,0.156378,-0.229519,NaN
203173,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-20,2010-10-20,27.09,TR,5872353.48,...,NaN,38351313.0,216772.0,0.023810,0.032046,0.047254,0.064951,0.156763,0.503859,NaN
203174,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-21,2010-10-21,27.58,TR,5978571.76,...,NaN,24173870.0,216772.0,0.018088,0.032004,0.047208,0.064975,0.157027,0.383150,NaN
203175,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-22,2010-10-22,27.54,TR,5969900.88,...,NaN,8143578.0,216772.0,-0.001450,0.032003,0.047197,0.064952,0.156966,-0.030729,NaN
203176,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-25,2010-10-25,27.95,TR,6058777.40,...,NaN,17871230.0,216772.0,0.014887,0.032007,0.047208,0.064977,0.157138,0.315356,NaN
203177,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-26,2010-10-26,27.73,TR,6011087.56,...,NaN,13787356.0,216772.0,-0.007871,0.032008,0.047201,0.064967,0.157163,-0.166760,NaN
203178,12346.0,LYB,53531.0,2821.0,325211.0,2010-10-27,2010-10-27,27.20,TR,5896198.40,...,NaN,2211360.0,216772.0,-0.019113,0.032003,0.047206,0.064978,0.156822,-0.404883,NaN


this still to be fixed


# Other Price Indicators

In [19]:
prices.sort_values(["permno", "date"], inplace=True)
prices

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,return,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return,naics_processed,industry_return,industry_relative_return,str_paper_residuals
0,10078,SUNW,8021,3570,NaN,2000-01-03,2000-01-03,76.500,TR,1.194246e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10078,SUNW,8021,3570,NaN,2000-01-04,2000-01-04,71.750,TR,1.120094e+08,...,-0.062092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10078,SUNW,8021,3570,NaN,2000-01-05,2000-01-05,71.875,TR,1.122045e+08,...,0.001742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10078,SUNW,8021,3570,NaN,2000-01-06,2000-01-06,68.000,TR,1.061552e+08,...,-0.053913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10078,SUNW,8021,3570,NaN,2000-01-07,2000-01-07,71.875,TR,1.122045e+08,...,0.056985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983884,93436,TSLA,53453,9999,336110.0,2023-12-22,2023-12-22,252.540,TR,8.028047e+08,...,-0.007701,0.044308,0.035495,0.030185,0.020859,-0.216973,33.0,0.002741,-0.219714,NaN
3983885,93436,TSLA,53453,9999,336110.0,2023-12-26,2023-12-26,256.610,TR,8.157429e+08,...,0.016116,0.044308,0.035025,0.030031,0.021031,0.460136,33.0,0.312875,0.147261,NaN
3983886,93436,TSLA,53453,9999,336110.0,2023-12-27,2023-12-27,261.440,TR,8.310971e+08,...,0.018822,0.044313,0.035013,0.030002,0.021262,0.537581,33.0,0.104966,0.432614,NaN
3983887,93436,TSLA,53453,9999,336110.0,2023-12-28,2023-12-28,253.180,TR,8.048392e+08,...,-0.031594,0.044319,0.034286,0.030136,0.020625,-0.921481,33.0,-0.000643,-0.920838,NaN


In [20]:
def calculate_MACD(data: pd.DataFrame, price_col: str):
    # Calculate the 12-period EMA
    data["EMA12"] = data[price_col].ewm(span=12, adjust=False).mean()

    # Calculate the 26-period EMA
    data["EMA26"] = data[price_col].ewm(span=26, adjust=False).mean()

    # Calculate MACD (the difference between 12-period EMA and 26-period EMA)
    data["MACD"] = data["EMA12"] - data["EMA26"]

    # Calculate the 9-period EMA of MACD (Signal Line)
    data["MACD_index"] = data["MACD"].ewm(span=9, adjust=False).mean()
    # data.loc[data['MACD'].isna(), 'MACD_index'] = np.nan

    mask = data["permno"] != data["permno"].shift(1)
    data.loc[mask, ["MACD", "MACD_index"]] = np.nan

    data.drop(["EMA12", "EMA26", "MACD"], axis=1, inplace=True)


calculate_MACD(prices, "DlyPrc")

In [21]:
prices

,permno,ticker,permco,SICCD,NAICS,date,DlyCalDt,DlyPrc,DlyPrcFlg,DlyCap,...,vol_36mo,vol_252d,vol_126d,vol_21d,vol_adjusted_return,naics_processed,industry_return,industry_relative_return,str_paper_residuals,MACD_index
0,10078,SUNW,8021,3570,NaN,2000-01-03,2000-01-03,76.500,TR,1.194246e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10078,SUNW,8021,3570,NaN,2000-01-04,2000-01-04,71.750,TR,1.120094e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.075783
2,10078,SUNW,8021,3570,NaN,2000-01-05,2000-01-05,71.875,TR,1.122045e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.192927
3,10078,SUNW,8021,3570,NaN,2000-01-06,2000-01-06,68.000,TR,1.061552e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.391237
4,10078,SUNW,8021,3570,NaN,2000-01-07,2000-01-07,71.875,TR,1.122045e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.567309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983884,93436,TSLA,53453,9999,336110.0,2023-12-22,2023-12-22,252.540,TR,8.028047e+08,...,0.044308,0.035495,0.030185,0.020859,-0.216973,33.0,0.002741,-0.219714,NaN,4.421231
3983885,93436,TSLA,53453,9999,336110.0,2023-12-26,2023-12-26,256.610,TR,8.157429e+08,...,0.044308,0.035025,0.030031,0.021031,0.460136,33.0,0.312875,0.147261,NaN,4.636163
3983886,93436,TSLA,53453,9999,336110.0,2023-12-27,2023-12-27,261.440,TR,8.310971e+08,...,0.044313,0.035013,0.030002,0.021262,0.537581,33.0,0.104966,0.432614,NaN,4.915273
3983887,93436,TSLA,53453,9999,336110.0,2023-12-28,2023-12-28,253.180,TR,8.048392e+08,...,0.044319,0.034286,0.030136,0.020625,-0.921481,33.0,-0.000643,-0.920838,NaN,5.076990


In [28]:
# RSI Index


def calculate_RSI(data: pd.DataFrame, price_col: str, window: int):
    change = data[price_col].diff()

    change_up, change_down = change.copy(), change.copy()

    change_up[change_up < 0] = 0
    change_down[change_down > 0] = 0

    # check we did not make mistakes
    assert change.equals(change_up + change_down)

    # calculate EWMAs
    avg_up = change_up.ewm(span=window, adjust=False).mean()
    avg_down = change_down.ewm(span=window, adjust=False).mean().abs()

    # calculate RSI
    rsi = 100 * avg_up / (avg_up + avg_down)
    data["rsi"] = rsi

    mask = data["permno"] != data["permno"].shift(1)
    data.loc[mask, "rsi"] = np.nan

In [29]:
calculate_RSI(prices, "DlyPrc", 14)